In [1528]:
import pandas as pd
import numpy as np
import os 

In [1529]:
cwd = os.getcwd()

df = pd.read_excel(cwd+"/B2B Data Provider-sample1.xlsx", index_col=0)

#replace all Nan values with -
df = df.replace(np.nan, "-")

In [1530]:
print(df.columns)

Index(['Starting Price - Monthly', 'Starting Price - Annually',
       'Pricing Range', 'Pricing comments', 'Additional Pricing Info',
       'Website', 'Contract', 'Solutions', 'Unlimited Data Plan',
       'CRM Integrations', 'SEPI', 'SalesTeamType', 'Free Trial',
       'ContactEnrichment', 'IntentData', 'Other', 'location'],
      dtype='object')


In [1531]:
#removing unnecessary columns
df_soft = df.drop(['Additional Pricing Info','Website','Unlimited Data Plan', 'Pricing Range', 'Pricing comments','Free Trial','Other'], axis=1)

#drop the pricing from data and this can be used to create a feature matrix
df_pro = df_soft.drop(['Starting Price - Monthly', 'Starting Price - Annually'], axis=1)
df_pro

,Contract,Solutions,CRM Integrations,SEPI,SalesTeamType,ContactEnrichment,IntentData,location
Company,,,,,,,,
ZoomInfo,Annual,"email,workphone,mobile","Microsoft Dynamics,HubSpot,Salesforce,Pipedriv...","Outreach,Salesloft","SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica
Cognism,Annual,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamics",Salesloft,"SMB,MidMarket,Enterprise",Yes,Yes,International
Uplead,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Insightly,Pipedrive,Zoho,Mi...","Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close","SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica
Lead411,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamic...",-,"SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica
Apollo,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket,Enterprise",Yes,No,USA/NorthAmerica
Seamless,Annual,"email,workphone,mobile","Salesforce,HubSpot,Microsoft Dynamics,Keap,Zoh...","Salesloft,Outreach","SMB,MidMarket",Yes,No,USA/NorthAmerica
RocketReach,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket",No,No,USA/NorthAmerica
Lusha,Annual or Monthly,"email,workphone","Salesforce,HubSpot","Salesloft,Outreach",SMB,Yes,No,USA/NorthAmerica
Crunchbase,Annual or Monthly,email,Salesforce,-,"SMB,MidMarket,Enterprise",No,No,USA/NorthAmerica


In [1532]:
#convert all the contract to list values
df_pro['Contract'] = df_pro['Contract'].str.replace(" or ", ",").str.lower().str.split(',')
#convert all solutions values to list 
df_pro['Solutions'] = df_pro['Solutions'].str.lower().str.split(",")
#convert all the CRM values to list
df_pro['CRM Integrations'] = df_pro['CRM Integrations'].str.lower().str.split(",")
#convert all SEPI values to list
df_pro['SEPI'] = df_pro['SEPI'].str.lower().str.split(",")
#convert all location values to list
df_pro['location'] = df_pro['location'].str.lower().str.split(",")
#convert all location values to list
df_pro['SalesTeamType'] = df_pro['SalesTeamType'].str.lower().str.split(",")

In [1533]:
def get_software_data_matrix(dataframe):

    for i, col in enumerate(dataframe.columns):

        for index,row in dataframe.iterrows():
            for contract in row[col]:
                if (dataframe.at[index,col] == "Yes") or (dataframe.at[index,col] == "No"):
                    pass
                else:
                    dataframe.at[index,contract] =1

    #replace the Yes and no values to 1 and 0 respectively for ContactEnrichment and IntentData columns
    dataframe = dataframe.replace('Yes', 1.0)
    dataframe = dataframe.replace('No', 0)


    #now drop all the initial columns that is not a matrix
    dataframe =dataframe.drop(['Contract','Solutions','CRM Integrations','SEPI','location', '-', 'SalesTeamType'], axis=1)

    #lastly replace all the nan values with 0
    dataframe= dataframe.replace(np.nan, 0)

    #lower casing all the column names
    dataframe.columns = dataframe.columns.str.lower()

    return dataframe

df_pro = get_software_data_matrix(df_pro)
df_pro

,contactenrichment,intentdata,annual,monthly,email,workphone,mobile,microsoft dynamics,hubspot,salesforce,...,salesloft,reply,woodpecker,lemlist,close,smb,midmarket,enterprise,usa/northamerica,international
Company,,,,,,,,,,,,,,,,,,,,,
ZoomInfo,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Cognism,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
Uplead,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
Lead411,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Apollo,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
Seamless,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
RocketReach,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
Lusha,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
Crunchbase,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0


In [1534]:
# #set the contract first
# for index,row in df_pro.iterrows():
#     for contract in row['Contract']:
#         df_pro.at[index,contract] =1

# #set the solutions 
# for index,row in df_pro.iterrows():
#     for sol in row['Solutions']:
#         df_pro.at[index,sol] =1
    
# #set the CRM
# for index,row in df_pro.iterrows():
#     for crm in row['CRM Integrations']:
#         df_pro.at[index,crm] = 1
        
# #set the Sales engagement matrix
# for index,row in df_pro.iterrows():
#     for sepi in row['SEPI']:
#         df_pro.at[index,sepi] = 1
        

# #set the location matric
# for index,row in df_pro.iterrows():
#     for loc in row['location']:
#         df_pro.at[index,loc] = 1
        
# #set the location matric
# for index,row in df_pro.iterrows():
#     for sales in row['SalesTeamType']:
#         df_pro.at[index,sales] = 1
        
# #now drop all the initial columns that is not a matrix
# df_pro =df_pro.drop(['Contract','Solutions','CRM Integrations','SEPI','location', '-', 'SalesTeamType'], axis=1)

# #lastly replace all the nan values with 0
# df_pro= df_pro.replace(np.nan, 0)


# #lower casing all the column names
# df_pro.columns = df_pro.columns.str.lower()

In [1535]:
#check the shape
df_pro.shape

(10, 35)

In [1536]:
#treated dataframe 
df_pro.columns

Index(['contactenrichment', 'intentdata', 'annual', 'monthly', 'email',
       'workphone', 'mobile', 'microsoft dynamics', 'hubspot', 'salesforce',
       'pipedrive', 'sugarcrm', 'zoho', 'insightly', 'copper', 'nimble',
       'infusionsoft', 'capsule', 'nutshell', 'base', 'vtiger', 'keap',
       'freshsales', 'freshworks', 'outreach', 'salesloft', 'reply',
       'woodpecker', 'lemlist', 'close', 'smb', 'midmarket', 'enterprise ',
       'usa/northamerica', 'international'],
      dtype='object')

**Now the idea is to cross multiply the values user values with and find the weighted values**

## USER data

In [1537]:
df_user = pd.read_csv("user_data.csv", index_col=0)
df_user

,data,contract,price,contacts,duration,Location,features,salesTeam,crm,SEPI_tools
0,"['email', 'workphone', 'mobile']","['Annual,Monthly']",50,500-1000,['Monthly'],['USA/NorthAmerica'],"['ContactEnrichment', 'IntentData']",['MidMarket'],"[""I don't have any CRM""]","['Woodpecker', 'Lemlist']"


In [1538]:
df_user.dtypes


data          object
contract      object
price          int64
contacts      object
duration      object
Location      object
features      object
salesTeam     object
crm           object
SEPI_tools    object
dtype: object

In [1539]:
for i, col in enumerate(df_user.columns):
    if df_user[col].dtype == "int64" or df_user[col].dtype == "float64":
        pass
    else:
        if col == "Location":
            if df_user['Location'].item().replace("'", "").strip("[]") == "Global":
                df_user['Location'] = [['usa/northamerica', 'international']]
            else:
                df_user[col] = df_user[col].item().replace("'", "").strip("[]").lower()
                df_user[col] = df_user[col].str.split(",")
        else:
            df_user[col] = df_user[col].item().replace("'", "").strip("[]").lower()
            df_user[col] = df_user[col].str.split(",")
        
        
    

In [1540]:
df_user

,data,contract,price,contacts,duration,Location,features,salesTeam,crm,SEPI_tools
0,"[email, workphone, mobile]","[annual, monthly]",50,[500-1000],[monthly],[usa/northamerica],"[contactenrichment, intentdata]",[midmarket],"[""i dont have any crm""]","[woodpecker, lemlist]"


In [1541]:
def get_user_data_matrix(dataframe):
    
    
    for i, col in enumerate(dataframe.columns):
        if col not in ['price','contacts']:
            if dataframe[col][0][0] == '"i dont have any crm"' or dataframe[col][0][0] == '"i dont have a sales engagement platform"':
                pass
            else:
                if col in ['crm', 'SEPI_tools']:
                    total_tools = len(dataframe['SEPI_tools'].item()) + len(dataframe['crm'].item())
                    for index,row in dataframe.iterrows():
                        for item in row[col]:
                            dataframe.at[index,item] = 1+(1/total_tools)
                else:
                    for index,row in dataframe.iterrows():
                        for item in row[col]:
                            dataframe.at[index,item] =1+(1/ len(dataframe[col]))
            
    
    
    #now drop all the initial columns that is not a matrix
    dataframe = dataframe.iloc[:,10:]

    #lastly replace all the nan values with 0
    dataframe= dataframe.replace(np.nan, 0)

    #lower casing all the column names
    dataframe.columns = dataframe.columns.str.lower()
    
    #lower casing all the column names
    dataframe.columns = dataframe.columns.str.strip(" ")
   
    
    if "crm" in dataframe.columns:
        dataframe.drop(['crm'], axis=1, inplace=True)
    
    if 'SEPI_tools' in dataframe.columns:
        dataframe.drop(['SEPI_tools'], axis=1, inplace=True)
        
    return dataframe


df_user_matrix = get_user_data_matrix(df_user)

In [1542]:

df_user_matrix

,email,workphone,mobile,annual,monthly,usa/northamerica,contactenrichment,intentdata,midmarket,woodpecker,lemlist
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.333333,1.333333


In [1543]:
#match columns and drop the ones that do not match
for i,item in enumerate(df_pro):
    if item not in df_user_matrix.columns:
         df_pro.drop(item, axis=1, inplace=True)
    else:
        for index in range(len(df_pro)):
            df_pro[item][index] = df_pro[item][index] * df_user_matrix[item][0]
        
#match columns and drop the ones that do not match
for i,item in enumerate(df_user_matrix):
    if item not in df_pro.columns:
        df_user_matrix.drop(item, axis=1, inplace=True)

#making a dot product between the two matrices to get the probability score 
#df_pro = df_pro.dot(df_user_matrix.T)

In [1544]:
df_pro.shape, df_user_matrix.shape

((10, 11), (1, 11))

In [1545]:
df_pro

,contactenrichment,intentdata,annual,monthly,email,workphone,mobile,woodpecker,lemlist,midmarket,usa/northamerica
Company,,,,,,,,,,,
ZoomInfo,2.0,2.0,2.0,0.0,2.0,2.0,2.0,0.000000,0.000000,2.0,2.0
Cognism,2.0,2.0,2.0,0.0,2.0,2.0,2.0,0.000000,0.000000,2.0,0.0
Uplead,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.333333,1.333333,2.0,2.0
Lead411,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.000000,0.000000,2.0,2.0
Apollo,2.0,0.0,2.0,2.0,2.0,2.0,2.0,0.000000,0.000000,2.0,2.0
Seamless,2.0,0.0,2.0,0.0,2.0,2.0,2.0,0.000000,0.000000,2.0,2.0
RocketReach,0.0,0.0,2.0,2.0,2.0,2.0,2.0,0.000000,0.000000,2.0,2.0
Lusha,2.0,0.0,2.0,2.0,2.0,2.0,0.0,0.000000,0.000000,0.0,2.0
Crunchbase,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.000000,0.000000,2.0,2.0


In [1546]:
df_pro.sum(axis=1)

Company
ZoomInfo       16.000000
Cognism        14.000000
Uplead         20.666667
Lead411        18.000000
Apollo         16.000000
Seamless       14.000000
RocketReach    14.000000
Lusha          12.000000
Crunchbase     10.000000
Snov.io         8.000000
dtype: float64

In [1547]:
df_soft['recommend']= df_pro.sum(axis=1)
#add back the pricing range column 
df_soft['Pricing Range'] = df['Pricing Range']


In [1548]:
if str(df_user['price'].item()) not in ["nan"]:
    if "monthly" in df_user['duration'].item():
        result = df_soft[df_soft['Starting Price - Monthly'] <= df_user['price'].item()].sort_values(by='Starting Price - Monthly', ascending=True)
    elif "annual" in df_user['duration'].item():
        result = df_soft[df_soft['Starting Price - Annually'] <= df_user['price'].item()].sort_values(by='Starting Price - Annually', ascending=True)
    else:
        result = df_soft.sort_values(by='Starting Price - Monthly', ascending=True)
else:
    result = df_soft.sort_values(by='recommend', ascending=False)

In [1549]:
result

,Starting Price - Monthly,Starting Price - Annually,Contract,Solutions,CRM Integrations,SEPI,SalesTeamType,ContactEnrichment,IntentData,location,recommend,Pricing Range
Company,,,,,,,,,,,,
ZoomInfo,0,11995,Annual,"email,workphone,mobile","Microsoft Dynamics,HubSpot,Salesforce,Pipedriv...","Outreach,Salesloft","SMB,MidMarket,Enterprise",Yes,Yes,USA/NorthAmerica,16.0,"Single user Pro Lite: $11,995 – 5,000 credits ..."
Cognism,0,16500,Annual,"email,workphone,mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamics",Salesloft,"SMB,MidMarket,Enterprise",Yes,Yes,International,14.0,"16,500-27,500+"
Seamless,0,1764,Annual,"email,workphone,mobile","Salesforce,HubSpot,Microsoft Dynamics,Keap,Zoh...","Salesloft,Outreach","SMB,MidMarket",Yes,No,USA/NorthAmerica,14.0,$1764 per year to $9528 per year+
Crunchbase,0,348,Annual or Monthly,email,Salesforce,-,"SMB,MidMarket,Enterprise",No,No,USA/NorthAmerica,10.0,348
Lusha,39,348,Annual or Monthly,"email,workphone","Salesforce,HubSpot","Salesloft,Outreach",SMB,Yes,No,USA/NorthAmerica,12.0,"$39 per monmth to $1,140 per year+"
Snov.io,39,396,Annual,email,"HubSpot,Salesforce,Pipedrive,Nutshell,Zoho,Fre...",-,"SMB,MidMarket",No,No,USA/NorthAmerica,8.0,"$39 per month to & 7,380"
Apollo,49,0,Annual or Monthly,"email,workphone,mobile","Salesforce,HubSpot","Salesloft,Outreach","SMB,MidMarket,Enterprise",Yes,No,USA/NorthAmerica,16.0,"$49 per month, month to month - $6000 per year+"


In [1550]:
# for i,item in enumerate(df_pro):
#     if item in df_user_matrix.columns:
#         print(item,i)
#         print(df_pro[item][0],df_user_matrix[item][0])
#         #df_pro[item][0] = df_pro[item][0] * df_user_matrix[item][0] 
#     else:
#         df_pro.drop(item, axis=1, inplace=True)
        
# df_pro